In [2]:
from torch import nn

In [3]:
# 문장 분류 모델 클래스 생성

class SentenceClassifier(nn.Module):
    
    def __init__(self, 
                 n_vocab,             # n_vocab: 단어 사전(vocabulary)의 크기 (총 단어 수)
                 hidden_dim,          # hidden_dim: RNN의 은닉 상태(hidden state)의 크기 (출력 차원)
                 embedding_dim,       # embedding_dim: 단어 임베딩(embedding) 벡터의 차원 (특징 수)
                 n_layers,            # n_layers: RNN 층의 개수 (몇 층으로 쌓을지 결정)
                 dropout=0.5,         # dropout: 드롭아웃(dropout) 비율 (과적합 방지를 위한 노드 무작위 비활성화 비율)
                 bidirectional=True,  # bidirectional: 양방향 RNN 여부 (True: 양방향, False: 단방향)
                 model_type="rnn"):
        super().__init__()
        
        
        # 임베
        self.embedding = nn.Embedding(num_embeddings=n_vocab,
                                      embedding_dim=embedding_dim,
                                      padding_idx=0)
        
        
        self.model = nn.RNN(input_size=embedding_dim,
                            hidden_size=hidden_dim,
                            num_layers=n_layers,
                            bidirectional=bidirectional,
                            dropout=dropout,
                            batch_first=True)
        
        
        if bidirectional:
            self.classifier = nn.Linear(hidden_dim * 2, 1)
        else:
            self.classifier = nn.Linear(hidden_dim,1)
        
        self.dropout = nn.Dropout(dropout)
        
        
    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        output, _ = self.model(embeddings)
        last_output = output[:,-1,:]
        last_output = self.dropout(last_output)
        logits = self.classifier(last_output)
        return logits

In [4]:
import pandas as pd
from Korpora import Korpora

corpus = Korpora.load("nsmc")
corpus_df = pd.DataFrame(corpus.test)

train = corpus_df.sample(frac=0.9, random_state=42)
test = corpus_df.drop(train.index)

print(train.head(5).to_markdown())
print('Training Data Size :', len(train))
print('Testing Data Size :', len(test))


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\knudc\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\knu

In [7]:
# 데이터 토큰화 및 단어 사전 구축
from konlpy.tag import Okt
from collections import Counter

def build_vocab(corpus, n_vocab, special_tokens):
    counter = Counter()
    
    for tokens in corpus:
        counter.update(tokens)
    
    vocab = special_tokens
    
    for token,count in counter.most_common(n_vocab):
        vocab.append(token)
    
    return vocab

In [ ]:
tokenizer = Okt()
train_tokens = [tokenizer.morphs(review) for review in train.text]
test_tokens = [tokenizer.morphs(review) for review in test.text]

vocab = build_vocab(corpus=train_tokens, n_vocab=5000, special_tokens=['<pad>','<unk>'])
token_to_id = {token:idx for idx,token in enumerate(vocab)}
id_to_token = {idx:token for idx,token in enumerate(vocab)}

print(vocab[:10])
print(len(vocab))  # 스페셜 토큰 + 5000개의 단어 사전 크기

['<pad>', '<unk>', '.', '이', '영화', '의', '..', '가', '에', '...']
5002
